In [1]:
import pandas as pd
import numpy as np
import altair as alt

In [2]:
from sklearn.feature_extraction.text import CountVectorizer
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

import re
from time import time

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\carlo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
df_relatos = pd.read_csv('C:\\Users\\carlo\\Downloads\\base_tratada(2019-2023).csv')


In [5]:
total_palavras = df_relatos['relato'].apply(lambda x: len(x.split())).sum()

print(f'Total de palavras no DataFrame: {total_palavras}')

Total de palavras no DataFrame: 6556784


In [6]:
df_relatos.shape

(47607, 21)

In [7]:
relatos=df_relatos[['relato']]

In [8]:
relatos['relato']

0        nacional marisangela ferreira pacheco uipp - t...
1        relatora qualificada , seccional urbana políci...
2        , especializada denunciar irmão ailton silva f...
3        relator qualificado , qualidade policial milit...
4        relatora qualificada epígrafe seccional comuni...
                               ...                        
47602    relatora qualificada comparece seccional polic...
47603    casada natanael dantas rocha junior , nascido ...
47604    relatora seccional policia civil sao bras comu...
47605    relatora qualificada comparece seccional polic...
47606    relatora qualificada comparece especializada c...
Name: relato, Length: 47607, dtype: object

* Pre-processamento

In [9]:
stop_words = set(stopwords.words("portuguese"))
stop_words.update(['fica', 'fim', 'ato', 'desde', 'pois', 'abadessa',
                   'abaetetuba', 'abaetetuba abaetetuba', 'abaetetuba contatos',
                   'abaixa', 'abaixada', 'abaixado', 'abaixar','abaixar som', 'abaixar volume',
                   'abaixasse', 'abaixasse som', 'abaixo', 'abaixo assinado','abaixo elencadas', 
                   'abaixo informacoes', 'abaixo olho','abaixo proibicao', 'abaixo proibição', 
                   'abaixou', 'abaixou pegar','abaixou vidro', 'relatora', 'policial', 'militar', 
                   'jurunas', 'silva', 'vtr', 'ciop', 'santos', 'bairro', 'fato','via', 'havia', 
                   'guarnicao', 'relator', 'sido', 'policia', 'sgt', 'rua','nacional', 'seccional', 
                   'icoaraci', 'urbana', 'seccional urbana', 'local', 'fato','silva', 'carlos', 'comunicar', 
                   'bairro','sacramenta', 'rua', 'qualificada seccional','marambaia', 'passagem', 'momento', 
                   'relatante','cpf', 'vou', 'textuais', 'vai', 'maria', 'vem', 'sra', 'depoente', 'relata', 
                   'jose', 'apos', 'nacional', 'local', 'celular', 'iria','relator', 'declarante', 'declarado', 
                   'policiais', 'guarnição', 'souza', 'antonio', 'paulo', 'santos', 'vou', 'textuais', 'qualificada',
                   'comparece'
])

stop_words = list(stop_words)


relatos_limpos = []
for w in range(len(relatos)):
  Relato = relatos['relato'].iloc[w]

  Relato  = re.sub("(\\d|\\W)+|\w*\d\w*"," ",Relato )
  Relato = ' '.join(s for s in Relato.split() if (not any(c.isdigit() for c in s)) and len(s) > 2)
  relatos_limpos.append(Relato)


relatos_limpos[0:5]

['nacional marisangela ferreira pacheco uipp terra firme comunicar citados trabalhando esposa funcinário trabalhar chamada nezia chaves gomes fazendo escândalo frente loja relatora trabalha mulher dizia vou pegar cortar toda cara desa puta vou deixar arreada chão furar todinha textuais vítima diz tempo nezia espalha relatora caso marido levy baia relatora trabalha sente constrangida fofocas ameaças tempo pede levy convencer esposa dois relação rapaz faz nada fica calado sabe suspeita reside marambaia passagem simão jatene número água cristal trabalha passagem francisco xavier final jabatiteua padaria fica esquina próximo vileta',
 'relatora qualificada seccional urbana polícia declarar enquanto visitava tia nacional geraldina brito sales hospital saúde mulher localizada humaita marco agredida fisicamente tapas arranhões puxões socos região rosto tórax membros superiores nacional simone carla sales duarte prima contenda começou devido problemas familiares relatora revidou procurou afast

* Processo de vetorização

In [11]:
tf_vectorizer = CountVectorizer(
    min_df=5,           # Considera palavras que aparecem em pelo menos 5 documentos
    max_df=0.5,          # Exclui termos que aparecem em mais de 50% dos documentos
    max_features=50000,  # Limita o número de termos no vocabulário aos 50.000 mais frequentes
    stop_words=stop_words, # Remove palavras comuns que não são úteis
    ngram_range=(1, 2)   # Considera uni-gramas (termos únicos) e bi-gramas (pares de termos)
)

#Transformação
vec_text = tf_vectorizer.fit_transform(relatos_limpos)

#Retorna a lista de palavras
words = tf_vectorizer.get_feature_names_out()

print(vec_text.shape)
print(len(words))

c:\Users\carlo\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\feature_extraction\text.py:408: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['assinado', 'contatos', 'elencadas', 'informacoes', 'olho', 'pegar', 'proibicao', 'proibição', 'som', 'vidro', 'volume'] not in stop_words.
  warnings.warn(


(47607, 50000)
50000


In [12]:
words[0:20]

array(['abala', 'abalada', 'abalada emocionalmente', 'abalada medo',
       'abalada psicologicamente', 'abalada situacao', 'abalada teme',
       'abalado', 'abalado psicologico', 'abalados', 'abalando', 'abalar',
       'abalo', 'abalo emocional', 'abalo psicologico', 'abalos',
       'abalos psicológicos', 'abalou', 'abandona', 'abandonada'],
      dtype=object)

* Encontrar os tópicos

In [13]:
from sklearn.decomposition import LatentDirichletAllocation

In [14]:
def print_top_words(model, feature_names, n_top_words):
  #Essa função imprime as palavras mais importantes para cada tópico descoberto pelo modelo LDA.
  for topic_idx, topic in enumerate(model.components_):
    print("\n--\nTopic #{}: ".format(topic_idx + 1))
    message = ", ".join([feature_names[i]
                          for i in topic.argsort()[:-n_top_words - 1:-1]])
    print(message)
  print()

def display_topics(W, H, feature_names, relatos, no_top_words, no_top_documents):
    #Essa função exibe os tópicos e os documentos mais representativos para cada tópico.
    for topic_idx, topic in enumerate(H):
        print("\n--\nTopic #{}: ".format(topic_idx + 1))
        print(", ".join([feature_names[i]
                for i in topic.argsort()[:-no_top_words - 1:-1]]).upper())
        top_d_idx = np.argsort(W[:,topic_idx])[::-1][0:no_top_documents]
        for d in top_d_idx:
            print('Relato índice {} : \t{:.2f}'.format(d, W[d, topic_idx]))

* Modelo de Latent Dirichlet Allocation (LDA) com a biblioteca scikit-learn para realizar modelagem de tópicos no conjunto de relatos representados por vetores de contagem de palavras.

In [15]:
lda = LatentDirichletAllocation(n_components=20, 
                                learning_method='batch', # 'online' equivale a minibatch no k-means
                                random_state=0)

t0 = time()

lda.fit(vec_text)
doc_topic_matrix = lda.transform(vec_text) #Converte os documentos representados pelos vetores vec_text em distribuições de tópicos, onde cada linha da matriz resultante (doc_topic_matrix) representa um documento e cada coluna representa a probabilidade do documento pertencer a um determinado tópico.

print("done in %0.3fs." % (time() - t0))

done in 1194.222s.


* Palavras mais associadas com cada tópico

In [16]:
print("\nTopics in LDA model:")
tf_feature_names = tf_vectorizer.get_feature_names_out()
print_top_words(lda, tf_feature_names, 20)


Topics in LDA model:

--
Topic #1: 
ameaça, vítima, seguinte, através, delegacia, comunica, mencionados, mencionados vítima, qualificado, virtual, comunica através, através delegacia, delegacia virtual, virtual mencionados, qualificado comunica, vítima ameaça, ameaça seguinte, dizendo, ameaças, pra

--
Topic #2: 
companheiro, medidas, protetivas, medidas protetivas, filho, deseja, contra, filhos, possui, casal, especializada, belém, filha, abrigo, relacionamento, nascido, criminalmente, criminalmente contra, meses, durante

--
Topic #3: 
agressor, ofendida, contato, proibicao, companheiro, filho, contra, proibicao agressor, aceita, medidas, solicita, protetivas, casal, medidas protetivas, quanto, proibição, criminalmente, testemunhas, quanto medidas, criminalmente contra

--
Topic #4: 
proibicao, ofendida, vitima, filho, medidas, protetivas, contra, medidas protetivas, familiares, especializada, testemunhas, telefone, ofendida familiares, relacao, agressor, familiares testemunhas, ame

* Documentos mais associados a cada topico

In [17]:
display_topics(doc_topic_matrix, lda.components_, tf_feature_names, relatos, 15, 10)


--
Topic #1: 
AMEAÇA, VÍTIMA, SEGUINTE, ATRAVÉS, DELEGACIA, COMUNICA, MENCIONADOS, MENCIONADOS VÍTIMA, QUALIFICADO, VIRTUAL, COMUNICA ATRAVÉS, ATRAVÉS DELEGACIA, DELEGACIA VIRTUAL, VIRTUAL MENCIONADOS, QUALIFICADO COMUNICA
Relato índice 12907 : 	0.99
Relato índice 12320 : 	0.99
Relato índice 15799 : 	0.99
Relato índice 15101 : 	0.99
Relato índice 13417 : 	0.99
Relato índice 12969 : 	0.99
Relato índice 14191 : 	0.99
Relato índice 11121 : 	0.99
Relato índice 17654 : 	0.99
Relato índice 11947 : 	0.99

--
Topic #2: 
COMPANHEIRO, MEDIDAS, PROTETIVAS, MEDIDAS PROTETIVAS, FILHO, DESEJA, CONTRA, FILHOS, POSSUI, CASAL, ESPECIALIZADA, BELÉM, FILHA, ABRIGO, RELACIONAMENTO
Relato índice 5775 : 	1.00
Relato índice 12344 : 	1.00
Relato índice 7675 : 	0.99
Relato índice 10504 : 	0.99
Relato índice 3174 : 	0.99
Relato índice 4145 : 	0.99
Relato índice 149 : 	0.99
Relato índice 4968 : 	0.99
Relato índice 17338 : 	0.99
Relato índice 8177 : 	0.99

--
Topic #3: 
AGRESSOR, OFENDIDA, CONTATO, PROIBICAO, CO

In [18]:
print('Matriz documento-tópicos:' + str(doc_topic_matrix.shape))
print('Matriz tópicos-termos:' + str(lda.components_.shape))

Matriz documento-tópicos:(47607, 20)
Matriz tópicos-termos:(20, 50000)


* Atribuir um tópico principal a cada documento com base nas distribuições de tópicos obtidas do modelo LDA.

In [19]:
main_topic = []
mt_prob = []
for l in range(len(relatos['relato'])):
  main_topic.append(doc_topic_matrix[l, :].argmax() + 1)
  mt_prob.append(doc_topic_matrix[l, :].max())

relatos = relatos.assign(main_topic=main_topic, main_topic_prob=mt_prob)


In [20]:
topico = 
pd.options.display.max_colwidth = 300
relatos[relatos['main_topic'] == topico].sort_values('main_topic_prob', ascending = False)[['main_topic_prob','relato']].head(20).sample(10)2

,main_topic_prob,relato
5729,0.989080,"informar ameaçada ex companheiro , valdinei lima ferreira paraense , nascido 28/03/1980 , ci.no . 3435229 , rodovia augusto montenegro número 11200 , tenoné , belém-pa , cep : 66820000 ; celular 998224-3442 ; valdinei conforma término relacionamento declarante vai ver capaz fazer ; declarante re..."
7492,0.992636,"especializada declarante sra . rosangela socorro ramos maciel , : 5347316 pc/pa , registrar conviveu união estável durante 12 sr. leonilson ferreira maciel , 40 , vigilante , avenida mangueirão , nº 74 , bairro : mangueirão , próximo bnar setor , belém/pa , possuindo filha menor 15 , especila po..."
17397,0.993066,"especializada declarante sra . rosilene santos almeida , 51 , : 1879072-6 via , pc/pa , registrar casada 18 conviveu união estável anteriormente casamento durante 04 sr. jose matos almeida , : 1345640-2 via pc/pa , filiação : luis pinheiro almeida adelaide matos santos , nascido capanema/pa 05/1..."
8177,0.993116,"especializada declarante sra . selma duarte gonçalves , 54 , : 2662383 pc/pa , registrar conviveu união estável durante 16 sr. rogério almeida sousa , 49 , rua japonês , quadra:14 , nº 13 , bairro : benguí , belém/pa , possuindo filhos desta união . ocorre separados ano ex-companheiro sr. rogéri..."
16771,0.992213,"especializada relatora sra . luciana saldanha santos , 42 , : 3309093 , segup/pa , registrar convive união estável 20 r. neycimar monteiro soares , 44 , endereço relatora sito travesa juvenal cordeuro , nº 585 , bairro : canudos , belém/pa , possuindo casal filhos 14 15 respectivamente desta uni..."
3174,0.994062,"especializada declarante sra . maria lair ribeiro maciel , 52 , : 1539336 pc/pa , registrar casada 36 sr. manoel anunciação rodrigues maciel , 52 , nascido 25/03/1967 , pedreiro , rua oliveira , nº 59 , bairro : 40 horas/ananindeua/pa , possuem três filhos desta relação todos maiores 33,32 30 re..."
4145,0.994025,"especializada declarante sra . jayne adriane araújo rodrigues , 20 , : 8137051 pc/pa , registrar conviveu união estável durante 06 sr. fabrício cesar correa silva , 21 , possui contato , travessa antonio baena , passagem freitas , nº 6-b , antonio baena pedro miranda , belém/pa , possuindo casal..."
4968,0.993581,"deseja registrar contra filho alesandro damasceno pereira , 37 , moto táxi , brasileiro , solteiro , nascido 07/08/1982 , : 3657097 ssp/pa , cpf : 696.080.752-20. passagem adriano , nº86 fundos , bairro : guamá . medidas protetivas urgência tomou ciência , processo nº0001374-96.2019.8.14.5150 1ª..."
12344,0.995178,"relatora identificada comunica delegacia especializada vítima ameaça parte ex companheiro nacional egídio pereira araújo neto , brasileiro , natural belém/pa , solteiro , vigilante , : 6603187-pc/pa , nascido 04/04/1987 , filho eliene nogueira araújo , rodovia augusto montenegro , rua joão nunes..."
5801,0.992910,"especializada declarante sra . luzia medeiros teles , 36 , : 4506940-pc/pa , registrar convive união estável 15 sr. fabio matos carvalho , 34 , desempregado , passagem amoras , nº 05 , rua mangueira franklin menezes , bairro : joão outeiro , belém/pa , possuindo 03 filhos desta união 14 , 12 10 ..."


* Tópicos por documento

In [21]:
# Nomes das colunas dos tópicos
topicnames = ["Topic {}".format(i + 1) for i in range(doc_topic_matrix.shape[1])]

# Criar DataFrame relato_topico
relato_topico = pd.DataFrame(np.round(doc_topic_matrix, 2), columns=topicnames, index=relatos.index)

# Adicionar o índice do relato ao DataFrame relato_topico
relato_topico['Relato_Index'] = relatos.index

# Reorganizar as colunas
ordem = ['Relato_Index']
ordem.extend(topicnames)
relato_topico = relato_topico[ordem]

# Visualizar o DataFrame relato_topico
relato_topico.head()



,Relato_Index,Topic 1,Topic 2,Topic 3,Topic 4,Topic 5,Topic 6,Topic 7,Topic 8,Topic 9,...,Topic 11,Topic 12,Topic 13,Topic 14,Topic 15,Topic 16,Topic 17,Topic 18,Topic 19,Topic 20
0,0,0.00,0.00,0.03,0.0,0.0,0.14,0.17,0.00,0.16,...,0.0,0.43,0.00,0.0,0.0,0.00,0.0,0.00,0.06,0.00
1,1,0.00,0.00,0.00,0.0,0.0,0.05,0.30,0.00,0.11,...,0.0,0.21,0.32,0.0,0.0,0.00,0.0,0.00,0.00,0.00
2,2,0.00,0.00,0.00,0.0,0.0,0.01,0.00,0.00,0.00,...,0.0,0.17,0.11,0.0,0.0,0.44,0.0,0.00,0.26,0.00
3,3,0.00,0.13,0.00,0.0,0.0,0.00,0.10,0.69,0.00,...,0.0,0.00,0.07,0.0,0.0,0.00,0.0,0.00,0.00,0.00
4,4,0.07,0.24,0.00,0.0,0.0,0.00,0.33,0.00,0.00,...,0.0,0.00,0.00,0.0,0.0,0.00,0.0,0.14,0.18,0.04


* Visualizações

In [22]:
import pyLDAvis
from pyLDAvis import lda_model
pyLDAvis.enable_notebook()

* Função prepare para preparar os dados para visualização de tópicos utilizando o modelo LDA.

In [23]:
pyLDAvis.lda_model.prepare(lda, vec_text, tf_vectorizer, sort_topics=False, mds = 'tsne')

PreparedData(topic_coordinates=               x          y  topics  cluster       Freq
topic                                                  
0     -20.402885  26.853191       1        1   2.498569
1      10.823973 -68.078224       2        1   6.880911
2      50.272854 -64.144966       3        1   4.152148
3      68.664909 -29.118431       4        1   6.015077
4      25.288008 -30.534100       5        1   3.956470
5     -11.377326 -33.208416       6        1   0.860578
6      24.863909  29.466433       7        1   4.712264
7      42.311012   0.571062       8        1   4.346486
8      38.321732  71.912369       9        1   6.962676
9       0.989623  53.162659      10        1   2.874369
10    -53.294312 -38.776772      11        1   6.108283
11    -59.638550  31.113510      12        1   5.069284
12     -3.534148  90.407791      13        1   5.591122
13    -40.137592  65.017563      14        1   5.287162
14      3.905545   0.266284      15        1   5.869124
15    -75.437561  -5.003512      16        1  12.800257
16     82.399757   8.264648      17        1   3.050275
17    -29.193821 -70.377464      18        1   4.375688
18    -34.160652  -4.637392      19        1   5.857060
19     62.445282  41.397484      20        1   2.732197, topic_info=            Term          Freq         Total Category  logprob  loglift
32786   ofendida  25061.000000  25061.000000  Default  30.0000  30.0000
20716      filha  21492.000000  21492.000000  Default  29.0000  29.0000
37296  proibicao  13513.000000  13513.000000  Default  28.0000  28.0000
1633    agressor  14614.000000  14614.000000  Default  27.0000  27.0000
49009     vitima  23474.000000  23474.000000  Default  26.0000  26.0000
...          ...           ...           ...      ...      ...      ...
32603   ocorrido    670.032077   4627.771450  Topic20  -5.4247   1.6676
19821      fatos    633.358646   4330.506925  Topic20  -5.4810   1.6777
3350         ana    595.937939   3344.873288  Topic20  -5.5419   1.8750
34058      parte    630.850870   7908.494904  Topic20  -5.4850   1.0714
29223      menor    603.581129   5307.687863  Topic20  -5.5291   1.4260

[1640 rows x 6 columns], token_table=       Topic      Freq                    Term
term                                          
111       16  0.998215  abrigamento temporario
114        2  0.193461                  abrigo
114        3  0.130044                  abrigo
114        4  0.163212                  abrigo
114        8  0.000195                  abrigo
...      ...       ...                     ...
49984      2  0.064899                  ônibus
49984      7  0.752242                  ônibus
49984     13  0.002950                  ônibus
49984     17  0.002950                  ônibus
49984     19  0.073749                  ônibus

[7787 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20])

* Transformando os dados para um espaço bidimensional, que pode ser utilizado para visualizações adicionais.

In [24]:
from sklearn.manifold import TSNE

relatos_embedded = TSNE(n_components=2, verbose=1, perplexity=40, early_exaggeration=20).fit_transform(doc_topic_matrix)

[t-SNE] Computing 121 nearest neighbors...
[t-SNE] Indexed 47607 samples in 0.003s...
[t-SNE] Computed neighbors for 47607 samples in 29.656s...
[t-SNE] Computed conditional probabilities for sample 1000 / 47607
[t-SNE] Computed conditional probabilities for sample 2000 / 47607
[t-SNE] Computed conditional probabilities for sample 3000 / 47607
[t-SNE] Computed conditional probabilities for sample 4000 / 47607
[t-SNE] Computed conditional probabilities for sample 5000 / 47607
[t-SNE] Computed conditional probabilities for sample 6000 / 47607
[t-SNE] Computed conditional probabilities for sample 7000 / 47607
[t-SNE] Computed conditional probabilities for sample 8000 / 47607
[t-SNE] Computed conditional probabilities for sample 9000 / 47607
[t-SNE] Computed conditional probabilities for sample 10000 / 47607
[t-SNE] Computed conditional probabilities for sample 11000 / 47607
[t-SNE] Computed conditional probabilities for sample 12000 / 47607
[t-SNE] Computed conditional probabilities for s

In [32]:
relatos = relatos.assign(tsne1 = relatos_embedded[:,0], tsne2 = relatos_embedded[:,1])
width = 600
height = 400
alt.Chart(relatos.sample(500)).mark_circle(
    opacity = .7,
    size = 30
).encode(
    x = 'tsne1',
    y = 'tsne2', 
    color = 'main_topic:N',
    size = 'main_topic_prob',
    tooltip = ['relato', 'main_topic', 'main_topic_prob']
).interactive().properties(
    width=width,
    height=height
)


alt.Chart(...)

* Agrupamento de acordo com os topicos

 Utilização do algoritmo MiniBatchKMeans para realizar clustering nos dados de tópicos obtidos a partir da matriz dos relatos transformados.

In [26]:
from sklearn.cluster import MiniBatchKMeans

kmeans = MiniBatchKMeans(n_clusters=11, init_size=1024, batch_size=2048, random_state=20)
#fit the data 
kmeans.fit(doc_topic_matrix)
labels = kmeans.predict(doc_topic_matrix)

In [31]:
relatos1 = relato_topico.assign(grupo = labels, 
                              tsne1 = relatos_embedded[:,0], tsne2 = relatos_embedded[:,1])
width = 600
height = 400
alt.Chart(relatos1.sample(500)).mark_circle(
    opacity = .7,
    size = 50
).encode(
    x = 'tsne1',
    y = 'tsne2', 
    color = 'grupo:N',
    tooltip = [str(c) for c in relatos1.columns]
).interactive().properties(
    width=width,
    height=height
)


alt.Chart(...)